## HLS版 最初のデバッグ4データ

In [231]:
import time

## FPGA Load
from pynq import Overlay
OL = Overlay("/home/xilinx/pynq/overlays/my_design/test_mnist_wrapper.bit")
OL.download()
XLNK = OL.processing_system7_0

## show IPs
print(OL.ip_dict.keys(), XLNK)

dict_keys(['axi_dma', 'axi_dma_0', 'axi_gpio_0', 'nn_0']) <pynq.xlnk.Xlnk object at 0xafd45230>


In [232]:
IMAGE_NUM = 4
## Allocate Memory
input_buf = XLNK.cma_array([28*28*IMAGE_NUM//4])
print(hex(input_buf.physical_address))
output_buf = XLNK.cma_array([1*IMAGE_NUM//4])
print(hex(output_buf.physical_address))

0x18050000
0x18052000


In [233]:
## Load Image
import re

def load_image(file_name, input_buf):
    with open(file_name, 'r') as f:
        val_s = 0
        for i, l in enumerate(f.readlines()):
            l = l.strip() ## remove return code
            l = re.sub("\s*\/\/.*$", "", l) ## remove Comment
            l = re.sub("\s+", " ", l) ## Remove Space 2+
            adr, val = l.split(" ")  ## split Space
            val = int(val, 16)

            val_s = val_s | (val << (8*(i%4)))
            if i%4==3:
                ## print(f"@{i//4}: 0x{val_s:08X}")
                input_buf[i//4] = val_s
                ## input_buf[i//4] = (i//4)
                val_s = 0

image_size = (28*28)//4
load_image("rom/image0.mem", input_buf[image_size*0:image_size*1]) ## 7
load_image("rom/image1.mem", input_buf[image_size*1:image_size*2]) ## 2
load_image("rom/image2.mem", input_buf[image_size*2:image_size*3]) ## 1
load_image("rom/image3.mem", input_buf[image_size*3:image_size*4]) ## 0


In [234]:
## Check Output Buf
for i in range(IMAGE_NUM//4):
    output_buf[i] = 0xFFFFFFFE
print(hex(output_buf[0]))

0xfffffffe


In [235]:
# OL.nn_0.s_axi_AXILiteS.register_map.CTRL.AP_START = 1
OL.nn_0.register_map.CTRL.AP_START = 1
OL.nn_0.register_map.CTRL.AUTO_RESTART = 1

In [236]:
# OL.nn_0.s_axi_AXILiteS.register_map.CTRL
OL.nn_0.register_map.CTRL

Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0)

In [237]:
## DMA Control
## Stop
OL.axi_dma_0.register_map.MM2S_DMACR = 0x0
OL.axi_dma_0.register_map.S2MM_DMACR = 0x0

## Run
OL.axi_dma_0.register_map.MM2S_DMACR = 0x1
OL.axi_dma_0.register_map.S2MM_DMACR = 0x1

## Address
OL.axi_dma_0.register_map.MM2S_SA = input_buf.physical_address
OL.axi_dma_0.register_map.S2MM_DA = output_buf.physical_address

## Size
OL.axi_dma_0.register_map.S2MM_LENGTH = 1*IMAGE_NUM
OL.axi_dma_0.register_map.MM2S_LENGTH = 28*28*IMAGE_NUM


In [238]:
# OL.nn_0.s_axi_AXILiteS.register_map.CTRL
for i in range(1):
    print(OL.nn_0.register_map.CTRL.__str__)
    time.sleep(1)

<bound method Register.__str__ of Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0)>


In [239]:
## Check Output Buf
print(f"0x{output_buf[0]:08X}")

0x00010207


In [240]:
## DMA MM2S Status, 
OL.axi_dma_0.register_map.MM2S_DMASR, OL.axi_dma_0.register_map.S2MM_DMASR

(Register(Halted=1, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
 Register(Halted=1, Idle=0, SGIncld=0, DMAIntErr=1, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=1, IRQThresholdSts=0, IRQDelaySts=0))

In [19]:
## DMA Soft Reset
OL.axi_dma_0.register_map.MM2S_DMACR = 0x4
OL.axi_dma_0.register_map.S2MM_DMACR = 0x4
OL.axi_dma_0.register_map.MM2S_DMACR, OL.axi_dma.register_map.MM2S_DMACR

(Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
 Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0))

---

# FPGA版 1転送 Loop

In [1]:
## FPGA Load
from pynq import Overlay
OL = Overlay("/home/xilinx/pynq/overlays/my_design/test_mnist_wrapper.bit")
OL.download()
XLNK = OL.processing_system7_0

## show IPs
print(OL.ip_dict.keys(), XLNK)

dict_keys(['axi_dma', 'axi_dma_0', 'axi_gpio_0', 'nn_0']) <pynq.xlnk.Xlnk object at 0xafd76210>


In [2]:
import mnist
import numpy as np
test_x = mnist.test_images()
test_y = mnist.test_labels()

test_x = test_x.reshape(10000, 28*28)

In [3]:
## Allocate Memory
IMAGE_NUM = 1
print(IMAGE_NUM)
input_buf = XLNK.cma_array([28*28*IMAGE_NUM], np.uint8)
print(hex(input_buf.physical_address))
output_buf = XLNK.cma_array([1*IMAGE_NUM], np.uint8)
print(hex(output_buf.physical_address))

1
0x18049000
0x1804a000


In [90]:
OL.nn_0.register_map.CTRL.AUTO_RESTART = 1
OL.nn_0.register_map.CTRL.AP_START = 1
OL.nn_0.register_map.CTRL

Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0)

In [117]:
import time

def copy_mem(n):
    ## Write inpu_buf(DDR)
    XLNK.cma_memcopy(input_buf, test_x[n], 28*28*1)    

def start_dma():
    ## DMA Control
    ## Stop
    OL.axi_dma_0.register_map.MM2S_DMACR = 0x4 ## TODO
    OL.axi_dma_0.register_map.S2MM_DMACR = 0x4 ## TODO

    ## Run
    OL.axi_dma_0.register_map.MM2S_DMACR = 0x1
    OL.axi_dma_0.register_map.S2MM_DMACR = 0x1

    ## Address
    OL.axi_dma_0.register_map.MM2S_SA = input_buf.physical_address
    OL.axi_dma_0.register_map.S2MM_DA = output_buf.physical_address

    ## Size
    OL.axi_dma_0.register_map.MM2S_LENGTH = 28*28*IMAGE_NUM
    OL.axi_dma_0.register_map.S2MM_LENGTH = 1*IMAGE_NUM

def wait_dma():
    i = 0
    while i<100:
        st = OL.axi_dma_0.register_map.S2MM_DMASR.Halted
        if st:
            break
        i += 1
        ## time.sleep(0.01)
    else:
        print("time out")


In [110]:
## DMA MM2S Status, 
OL.axi_dma_0.register_map.MM2S_DMASR, OL.axi_dma_0.register_map.S2MM_DMASR

(Register(Halted=1, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
 Register(Halted=1, Idle=0, SGIncld=0, DMAIntErr=1, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=1, IRQThresholdSts=0, IRQDelaySts=0))

In [111]:
OL.axi_dma_0.register_map.S2MM_LENGTH

Register(Length=1)

In [112]:
output_buf[0] = 0xFF

copy_mem(0)
start_dma()

In [113]:
output_buf[0], hex(output_buf[0])

(7, '0x7')

#### DMAIntErr=1
- どうもTLASTを使ってないから(0固定)、エラーが立つらしい。
  - [AXIDMAエラー-コミュニティフォーラム](https://forums.xilinx.com/t5/Embedded-Development-Tools/AXI-DMA-Error/td-p/226325)

- エラー時は、次の転送を開始してくれないぽい。
  → 毎回リセット入れる。

In [114]:
## DMA MM2S Status, 
OL.axi_dma_0.register_map.MM2S_DMASR, OL.axi_dma_0.register_map.S2MM_DMASR

(Register(Halted=1, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
 Register(Halted=1, Idle=0, SGIncld=0, DMAIntErr=1, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=1, IRQThresholdSts=0, IRQDelaySts=0))

In [92]:
## DMA Soft Reset
OL.axi_dma_0.register_map.MM2S_DMACR = 0x4
OL.axi_dma_0.register_map.S2MM_DMACR = 0x4
OL.axi_dma_0.register_map.MM2S_DMACR, OL.axi_dma_0.register_map.MM2S_DMACR

(Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
 Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0))

In [135]:
%%time
N = 100 ## 2min
# N = 1000 ## 2min
ok = 0
for n in range(N):
    copy_mem(n)
    start_dma()
    wait_dma()
    if test_y[n] == output_buf[0]:
        ok += 1
    ## print(output_buf[0])
    print(f"\r{n+1}/{N}: {ok/(n+1):.4}", end='')
print()
print(f"{ok/N:.4}")

100/100: 0.977
0.97
CPU times: user 11.2 s, sys: 1.66 s, total: 12.8 s
Wall time: 12 s


### WeightとBiasをRAMにしていたときの名残。
→ Layer0.b1に、アクセスすると、Jupyterがハングするので、ROMにするようにした。




In [18]:
l = (1,2)
import numpy as np
n = np.arange(10)
", ".join([str(nn) for nn in n.tolist()])

'0, 1, 2, 3, 4, 5, 6, 7, 8, 9'

In [2]:
def write_mmio(file_name, mmio, offset):
    ss2int = lambda s: int(s, 16)
    with open(file_name, 'r') as f:
        for i, l in enumerate(f.readlines()):
            if i%4==0:
                v = ss2int(l)
            elif i%4==1:
                v = v | (ss2int(l)<<8)
            elif i%4==2:
                v = v | (ss2int(l)<<16)
            elif i%4==3:
                v = v | (ss2int(l)<<24)
#                 print(hex(v))
                mmio.write(offset + i-3, v)


In [3]:
from pynq import MMIO
nn = OL.ip_dict['nn_0']
base_adr = nn['phys_addr']
mmio = MMIO(base_adr, 0x2480f)

hex(mmio.length)
mmio.read(0), mmio.read(4), mmio.read(0x10000)

(4, 0, 0)

In [ ]:
## Weight, BiasをLoad

from pynq import MMIO

nn = OL.ip_dict['nn_0']
base_adr = nn['phys_addr']
mmio = MMIO(base_adr, 0x2480f)


## Layer0, W
size = 28*28
offset = nn['registers']['Memory_w0']['address_offset']
for i in range(50):
    write_mmio(f"rom_hls/W1_{i}.mem", mmio, offset + size*i)

## Layer0, B
size = 50
offset = nn['registers']['Memory_b0']['address_offset']
write_mmio(f"rom_hls/b1.mem", mmio, offset)

# ## Layer1, W
# size = 50
# offset = nn['registers']['Memory_w1']['address_offset']
# for i in range(100):
#     write_mmio(f"rom_hls/W2_{i}.mem", MMIO(base_adr+offset + size*i, size))
    
# ## Layer1, B
# size = 100
# offset = nn['registers']['Memory_b1']['address_offset']
# write_mmio(f"rom_hls/b2.mem", MMIO(base_adr+offset, size))

# ## Layer2, W
# size = 100
# offset = nn['registers']['Memory_w2']['address_offset']
# for i in range(10):
#     write_mmio(f"rom_hls/W3_{i}.mem", MMIO(base_adr+offset + size*i, size))

# ## Layer2, B
# size = 10
# offset = nn['registers']['Memory_b2']['address_offset']
# write_mmio(f"rom_hls/b3.mem", MMIO(base_adr+offset, size))


In [4]:
hex(mmio.base_addr)

'0x80022000'

In [ ]:
mmio.read_mm()